In [1]:
import os
import re

In [2]:
import pymorphy2
from nltk.tokenize import TweetTokenizer

morph = pymorphy2.MorphAnalyzer()
tok = TweetTokenizer(preserve_case=False)

In [3]:
wordset = {w.strip() for w in open('alphabet_rus.txt', encoding='cp1251')}

In [17]:
from multiprocessing import Pool

def process_word(word):
    if word.isalnum():
        if morph.parse(word)[0].normal_form in wordset:
            return word


pattern = re.compile('<div class=\"text-block\">([\s\S]+)</div>')

texts = []
empty = []

p = Pool(processes=4)

for d in os.listdir('data'):
    for file in os.listdir(os.path.join('data', d)):
        path = os.path.join('data', d, file)
        with open(path) as f:
            data = f.read()
        text = pattern.search(data).group(1).strip()
        words = p.map(process_word, tok.tokenize(text))
        flag = False
        for word in words:
            if word is not None:
                text = text.replace(" " + word, " " + word.upper())
                flag = True
        if flag:
            texts.append(text)
        else:
            empty.append(text)
p.close()

In [21]:
with open('found.txt', 'w') as f:
    f.write('\n-/-\n'.join(texts))

In [16]:
with open('rest.txt', 'w') as f:
    f.write('\n-/-\n'.join(empty))

In [22]:
texts[0]

'За последние сутки в России выявлено 7 600 заболевших новой коронавирусной инфекцией в 83 регионах, из них 2 473 случая (32,5%) – активно у контактных лиц без клинических проявлений болезни❗️\n\nВсего в стране нарастающим итогом выявлено 592 280 заболевших в 85 регионах. \n\nТакже за сутки зафиксировано 95 летальных случаев. За ВЕСЬ период в России от коронавируса скончались 8 206 человек.\n\nКарта заболеваемости коронавирусной инфекцией с подробностями по регионам и динамикой по дням – на сайте стопкоронавирус.рф\n\n#стопкоронавирус #здоровьевприоритете #coronavirus'